In [12]:
import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,
        'options': {
        'defaultType': 'spot',  }
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

In [16]:
scrape_candles_to_csv('btc_usdt.csv', 'binance', 5, 'BTC/USDT', '1h', '2017-01-0100:00:00Z', 500)

500 BTC/USDT candles in total from 2022-02-25T01:00:00.000Z to 2022-03-17T20:00:00.000Z
1000 BTC/USDT candles in total from 2022-02-04T05:00:00.000Z to 2022-03-17T20:00:00.000Z
1500 BTC/USDT candles in total from 2022-01-14T09:00:00.000Z to 2022-03-17T20:00:00.000Z
2000 BTC/USDT candles in total from 2021-12-24T13:00:00.000Z to 2022-03-17T20:00:00.000Z
2500 BTC/USDT candles in total from 2021-12-03T17:00:00.000Z to 2022-03-17T20:00:00.000Z
3000 BTC/USDT candles in total from 2021-11-12T21:00:00.000Z to 2022-03-17T20:00:00.000Z
3500 BTC/USDT candles in total from 2021-10-23T01:00:00.000Z to 2022-03-17T20:00:00.000Z
4000 BTC/USDT candles in total from 2021-10-02T05:00:00.000Z to 2022-03-17T20:00:00.000Z
4500 BTC/USDT candles in total from 2021-09-11T09:00:00.000Z to 2022-03-17T20:00:00.000Z
5000 BTC/USDT candles in total from 2021-08-21T13:00:00.000Z to 2022-03-17T20:00:00.000Z
5500 BTC/USDT candles in total from 2021-07-31T17:00:00.000Z to 2022-03-17T20:00:00.000Z
6000 BTC/USDT candles 